In [2]:
import pandas as pd
import numpy as np
import math
import random
from tabulate import tabulate
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split

In [3]:
# Leitura dos dados
def leituraDados():
  url = "https://raw.githubusercontent.com/RafaelBortolozo/Pattern_recognition/main/Raisin_Dataset.csv"
  dataset = pd.read_csv(url, sep=",")

  X = dataset.columns[0:7]
  X = dataset[X]

  y = dataset["Class"]

  # display(X)
  # display(pd.DataFrame(y, columns = ['Class']))

  return X, y

In [4]:
# Normalização (desvio padrão, min-max)
def normalizacao(data, method="minmax"):
  new_data = []

  if method == "std":
    data_T = X.T
    for i in data_T:
      new_data.append((i - np.mean(row)) / ((np.std(i, ddof=1))**2))
    new_data = np.array(new_data).T

  if method == "minmax":
    new_data = (data - data.min(axis=0)) / (data.max(axis=0) - data.min(axis=0))

  return new_data

In [5]:
# "Kecimen" = 0
# "Besni"   = 1
def mapeamentoClasses(df):
  data = []
  for i in df:
    if i == "Kecimen":
      data.append(0)
    elif i == "Besni":
      data.append(1)
  return data

In [6]:
def print_resultado(name, variance, explained_variance_ratio, n_components, acc, cm, tpr, tnr):
  print('==================================================')
  print(f'Modelo: {name}')
  print(f'Variância acumulada: {variance} -> {explained_variance_ratio}')
  print(f'Quantidade CPs: {n_components}')
  print(f'Acurácia: {round(acc,2)}')
  print(f'TPR: {round(tpr,2)}')
  print(f'TNR: {round(tnr,2)}')
  print(f'Matriz de confusão:')
  print(f"{cm[0]}\n{cm[1]}")

In [7]:
def dividirDataset(data,labels,test_size=0.33,shuffle=True):
   X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=test_size, random_state=random.randint(10,48),shuffle=shuffle) 
   return X_train, X_test, y_train, y_test

In [8]:
def treinar_modelo(X, y, pca, model, name):
  novo_dataset = pca.fit_transform(np.copy(X), y)
  
  X_train, X_test, y_train, y_test = train_test_split(novo_dataset, y, test_size=0.33, random_state=16)
  
  model.fit(X_train, y_train)
  
  pred = model.predict(X_test)
  
  cm = confusion_matrix(y_test, pred)
  tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
  acc = (tn + tp) / (tn + fp + fn + tp)
  tpr = tp / (tp + fn)
  tnr = tn / (tn + fp)
  accumulated_variance = round(sum(pca.explained_variance_ratio_), 2)
  explained_variance_ratio = [round(item, 2) for item in pca.explained_variance_ratio_]

  print_resultado(name, accumulated_variance, explained_variance_ratio, pca.n_components_, acc, cm, tpr, tnr)

In [10]:
X, y = leituraDados()
X = normalizacao(X, "minmax")
y = mapeamentoClasses(y)


for num_cps in [1, 2, 7]:
  print("\n")
  pca = PCA(n_components = num_cps, svd_solver='auto')

  treinar_modelo(
      X, y, pca, GaussianNB(), 'Naive Bayes Gaussiano'
  );

  treinar_modelo(
      X, y, pca, SVC(kernel='linear'), 'SVM Linear'
  );
  
  treinar_modelo(
      X, y, pca, SVC(kernel='rbf'), 'SVM Não Linear (RBF)'
  );

  treinar_modelo(
      X, y, pca, DecisionTreeClassifier(splitter='random'), 'C4.5 - CART'
  );

  print("\n")



Modelo: Naive Bayes Gaussiano
Variância acumulada: 0.75 -> [0.75]
Quantidade CPs: 1
Acurácia: 0.85
TPR: 0.74
TNR: 0.96
Matriz de confusão:
[135   5]
[ 41 116]
Modelo: SVM Linear
Variância acumulada: 0.75 -> [0.75]
Quantidade CPs: 1
Acurácia: 0.86
TPR: 0.81
TNR: 0.91
Matriz de confusão:
[127  13]
[ 30 127]
Modelo: SVM Não Linear (RBF)
Variância acumulada: 0.75 -> [0.75]
Quantidade CPs: 1
Acurácia: 0.86
TPR: 0.81
TNR: 0.92
Matriz de confusão:
[129  11]
[ 30 127]
Modelo: C4.5 - CART
Variância acumulada: 0.75 -> [0.75]
Quantidade CPs: 1
Acurácia: 0.73
TPR: 0.74
TNR: 0.71
Matriz de confusão:
[100  40]
[ 41 116]




Modelo: Naive Bayes Gaussiano
Variância acumulada: 0.92 -> [0.75, 0.17]
Quantidade CPs: 2
Acurácia: 0.88
TPR: 0.8
TNR: 0.96
Matriz de confusão:
[135   5]
[ 32 125]
Modelo: SVM Linear
Variância acumulada: 0.92 -> [0.75, 0.17]
Quantidade CPs: 2
Acurácia: 0.87
TPR: 0.82
TNR: 0.93
Matriz de confusão:
[130  10]
[ 29 128]
Modelo: SVM Não Linear (RBF)
Variância acumulada: 0.92 -> [0.7